In [6]:
from __future__ import print_function
import pyodbc
import math
import numpy as np
import os
import sys
from sklearn.cluster import KMeans

In [35]:
conn = pyodbc.connect(r'DRIVER={ODBC Driver 13 for SQL Server};'
                      r'SERVER=mpk4uitxo6.database.windows.net;'
                      r'DATABASE=Sensoria.Knowledge.Database.Live;'
                      r'UID=liveuser;'
                      r'PWD=7oHWE^Gcr834C*zH;')

In [8]:
cursor=conn.cursor()
cursor.execute("WITH cte AS(SELECT SessionId,[TelemetryTimeTick] AS Tick,GeoLocation as P1, LEAD(GeoLocation, 1) OVER (PARTITION BY SessionId ORDER BY [TelemetryTimeTick]) as P2,LEAD([TelemetryTimeTick], 1) OVER (PARTITION BY SessionId ORDER BY [TelemetryTimeTick]) as T2 FROM  dbo.[Sessions.TelemetryData]), cte2 as (select SessionId,Tick,DistanceCalculated = P1.STDistance(P2),TimeDistance = T2-Tick,P1.STDistance(P2)/(T2-Tick) as SpeedCalculated from cte	), cte3 as( select	*,PointClass = case when DistanceCalculated is null or SpeedCalculated < 0.85 then 'A' when SpeedCalculated > 11 then 'B' else 'C' end from cte2), cte4 as( select	SessionId,CountA = count(case when PointClass = 'A' then 1 else null end),CountC = count(case when PointClass = 'C' then 1 else null end),CountB = count(case when PointClass = 'B' then 1 else null end),DistanceB = sum(case when PointClass = 'B' then DistanceCalculated else null end),TotalDistance = sum(DistanceCalculated),TotalCount = count(*) from cte3 group by SessionId)select SessionId,(CountA*1.)/TotalCount,(CountC*1.)/TotalCount,(CountB*1.)/TotalCount,(DistanceB * 1.) / NULLIF(TotalDistance,0) from cte4")
rows= cursor.fetchall()

In [9]:
arrayMultiDim=np.array([])
sessionId=list()
for i in range(len(rows)):
    if rows[i][1] is not None:
        arrayMultiDim=np.append(arrayMultiDim,float(rows[i][1]))
    else:
        arrayMultiDim=np.append(arrayMultiDim,0)
    if rows[i][2] is not None:
        arrayMultiDim=np.append(arrayMultiDim,float(rows[i][2]))
    else:
        arrayMultiDim=np.append(arrayMultiDim,0)
    if rows[i][3] is not None:
        arrayMultiDim=np.append(arrayMultiDim,float(rows[i][3]))
    else:
        arrayMultiDim=np.append(arrayMultiDim,0)
    if rows[i][4] is not None:
        arrayMultiDim=np.append(arrayMultiDim,float(rows[i][4]))
    else:
        arrayMultiDim=np.append(arrayMultiDim,0)
    sessionId.append(int(rows[i][0]))
arrayMD=arrayMultiDim.reshape(len(rows),4)

In [11]:
len(sessionId),len(arrayMD)

(23431, 23431)

In [44]:
#for i in range(len(sessionId)):
#    if sessionId[i]==27650:
#        print(km.labels_[i])

0


In [12]:
km=KMeans(n_clusters=3).fit(arrayMD)
#centroidi
print(km.cluster_centers_)
km.predict((np.array([0.500, 0.500, 0.000, 0.000 ])).reshape(1,4))

[[  9.79422407e-01   2.03286273e-02   2.48966141e-04   7.75551369e-03]
 [  9.83361062e-02   8.99103804e-01   2.56008982e-03   2.01184286e-02]
 [  5.44055343e-01   1.04783829e-01   3.51160828e-01   8.76390807e-01]]


array([1])

In [13]:
km=KMeans(n_clusters=5).fit(arrayMD)
#centroidi
print(km.cluster_centers_)

[[  4.92549900e-01   4.97413798e-01   1.00363021e-02   7.75086267e-02]
 [  5.85317218e-02   9.39508313e-01   1.95996564e-03   1.48826491e-02]
 [  9.90121613e-01   9.80391641e-03   7.44706429e-05   3.52583213e-03]
 [  2.22004013e-01   4.41361651e-02   7.33859822e-01   9.77085914e-01]
 [  7.82206229e-01   9.62466570e-02   1.21547114e-01   8.02177260e-01]]


In [14]:
km=KMeans(n_clusters=4).fit(arrayMD)
#centroidi
print(km.cluster_centers_)

[[  9.89692032e-01   1.01304753e-02   1.77492668e-04   7.19140306e-03]
 [  5.86197850e-02   9.39501536e-01   1.87867916e-03   1.44940921e-02]
 [  5.38904096e-01   7.64500560e-02   3.84645848e-01   9.18006025e-01]
 [  4.97133821e-01   4.92585957e-01   1.02802212e-02   8.45980117e-02]]


In [19]:
for i in range(len(rows)):
    if km.labels_[i]==1 and arrayMD[i][3]>0.1:
        km.labels_[i]=2
#casi anomali

In [20]:
import pandas as pd
df=pd.DataFrame()
df['Id'] = sessionId
df['labels']=km.labels_

In [21]:
df

,Id,labels
0,9737,0
1,9739,0
2,9741,0
3,9743,0
4,9745,0
5,9747,0
6,15095,0
7,15096,0
8,16137,0
9,16258,0


In [18]:
df.to_csv('ClassificationNew')